## Литература

https://webdevblog.ru/tematicheskoe-modelirovanie-s-pomoshhju-gensim-python/

In [1]:
!pip install nltk
!pip install pymorphy2 -q
!pip install geopy -q

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 55.5/55.5 kB 1.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 8.2/8.2 MB 17.9 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done


In [2]:
!pip install numpy==1.23.5
!pip install pandas==1.5.3

In [3]:
!pip install funcy
!pip install tzdata

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 346.6/346.6 kB 5.0 MB/s eta 0:00:00


In [4]:
!pip install --no-deps pyLDAvis==3.4.0

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.6/2.6 MB 18.7 MB/s eta 0:00:00


In [5]:
import nltk

import pandas as pd
from collections import defaultdict
from geopy.geocoders import Nominatim
import numpy as np
import time

from nltk.corpus import stopwords
import pymorphy2
import re
import os

In [6]:
from sklearn.feature_extraction.text import CountVectorizer, TfidfVectorizer
from sklearn.decomposition import LatentDirichletAllocation

In [7]:
import pyLDAvis
import pyLDAvis.lda_model

In [8]:
from google.colab import drive
drive.mount('/content/drive')

/usr/local/lib/python3.10/dist-packages/ipykernel/ipkernel.py:283: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)


Mounted at /content/drive


In [9]:
os.chdir('/content/drive/MyDrive/Colab Notebooks/Авалон/Диплом')

/usr/local/lib/python3.10/dist-packages/ipykernel/ipkernel.py:283: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)


In [10]:
nltk.download('stopwords')

/usr/local/lib/python3.10/dist-packages/ipykernel/ipkernel.py:283: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)
[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.


True

In [11]:
df = pd.read_csv('data/data.csv')
df['rating'] = df['rating'].astype('float16')

/usr/local/lib/python3.10/dist-packages/ipykernel/ipkernel.py:283: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)


In [12]:
# на основе выделенных словарей по рубрикам - лучше анализировать рестораны и кафе
interesting_rub_list = ['Кафе', 'Ресторан', 'Бар, паб', 'Быстрое питание', 'Кофейня', 'Пиццерия',
                        'Пекарня', 'Столовая', 'Суши-бар', 'Кофе с собой']

# coffee = df[df['rubrics'].isin(interesting_rub_list)] только уникальные включения ("Кафе, Кофейня" не будет)
mask = df['rubrics'].apply(lambda x: True if any(rub_x in x for rub_x in interesting_rub_list) else False)
coffee = df[mask]
coffee.index = np.arange(coffee.shape[0])
coffee.info()

/usr/local/lib/python3.10/dist-packages/ipykernel/ipkernel.py:283: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)


<class 'pandas.core.frame.DataFrame'>
Int64Index: 117818 entries, 0 to 117817
Data columns (total 5 columns):
 #   Column   Non-Null Count   Dtype  
---  ------   --------------   -----  
 0   address  117818 non-null  object 
 1   name_ru  117587 non-null  object 
 2   rating   117818 non-null  float16
 3   rubrics  117818 non-null  object 
 4   text     117818 non-null  object 
dtypes: float16(1), object(4)
memory usage: 4.7+ MB


## Функции

In [13]:
def preprocess_text(data, stop_words):
    """Очистка текста и удаление стоп-слов"""
    # замена ё на е // оставляем буквы, пробелы и знаки препинания // удаление пробелов
    text = re.sub('ё', 'е', data.lower())
    text = re.sub(r"([.,!?])", r'\1', text)
    text =re.sub(r"[^а-яА-Я\s]+", r'', text)
    text = text.strip()

    # удаление стоп слов и слов меньше длины 3
    stop_words.extend(['очень', 'самый', 'самое', 'самая'])
    text = [w for w in text.split() if w not in stop_words]
    text = [w for w in text if len(w) >= 3]

    return ' '.join(text)

def lemmatization_text(data, morph):
    result = " ".join([morph.parse(word_x)[0].normal_form
                      for word_x in data.split()])
    return result

def get_result(data, morph=pymorphy2.MorphAnalyzer(), stop_words=stopwords.words("russian")):
    if data is not np.NaN:
        result = preprocess_text(data=data, stop_words=stop_words)
        result = lemmatization_text(data=result, morph=morph)
        return result
    return ""

def transform_data(data: pd.Series) -> list:
  transform_lst = [get_result(data=i) for i in data]
  transform_lst = [x for x in transform_lst if x not in [None, '']]
  return transform_lst

/usr/local/lib/python3.10/dist-packages/ipykernel/ipkernel.py:283: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)
/usr/local/lib/python3.10/dist-packages/pymorphy2/units/base.py:70: DeprecationWarning: inspect.getargspec() is deprecated since Python 3.0, use inspect.signature() or inspect.getfullargspec()
  args, varargs, kw, default = inspect.getargspec(cls.__init__)
/usr/local/lib/python3.10/dist-packages/pymorphy2/units/base.py:70: DeprecationWarning: inspect.getargspec() is deprecated since Python 3.0, use inspect.signature() or inspect.getfullargspec()
  args, varargs, kw, default = inspect.getargspec(cls.__init__)
/usr/local/lib/python3.10/dist-packages/pymorphy2/units/base.py:70: DeprecationWarning: inspect.getargspec() is deprecated since Py

In [14]:
pyLDAvis.enable_notebook()
def theme_class(input_data, vectorizer_x=None, model_type=None):
  result = transform_data(input_data)

  if vectorizer_x is None:
    vectorizer_x = CountVectorizer(analyzer='word',
                                 min_df=10,
                                 ngram_range=(2,3),
                                 stop_words=stopwords.words("russian"))

  res_vectorizer = vectorizer_x.fit_transform(result)

  if model_type is None:
    model_type = LatentDirichletAllocation(n_components=5,
                                          learning_method='online',
                                          random_state=42,
                                          n_jobs=-1)
  model_type.fit(res_vectorizer)


  panel = pyLDAvis.lda_model.prepare(model_type,
                                     res_vectorizer,
                                     vectorizer,
                                     mds='tsne')
  print("Log Likelihood", model_type.score(res_vectorizer))
  print("Perplexity", model_type.perplexity(res_vectorizer))

  return panel, model_type, vectorizer_x

def show_topics( vectorizer_x=None, model=None, n_words=20):
  feature_names = np.array(vectorizer_x.get_feature_names_out())
  top_words = []

  for topic_weights in model.components_:
    top_keywords_locs = (-topic_weights).argsort()[:n_words]
    top_words.append(feature_names.take(top_keywords_locs))
  return top_words

/usr/local/lib/python3.10/dist-packages/ipykernel/ipkernel.py:283: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)


In [15]:
start = time.time()
# на основе выделенных словарей по рубрикам - лучше анализировать рестораны и кафе
interesting_rub_list = ['Кафе', 'Ресторан', 'Бар, паб', 'Быстрое питание', 'Кофейня', 'Пиццерия',
                        'Пекарня', 'Столовая', 'Суши-бар', 'Кофе с собой']

# coffee = df[df['rubrics'].isin(interesting_rub_list)] только уникальные включения ("Кафе, Кофейня" не будет)
mask = df['rubrics'].apply(lambda x: True if any(rub_x in x for rub_x in interesting_rub_list) else False)
coffee = df[mask]
coffee = coffee[coffee['rating'] <=3]
coffee.index = np.arange(coffee.shape[0])

vectorizer = TfidfVectorizer(analyzer='word',
#vectorizer = CountVectorizer(analyzer='word',
                             # binary=True,    # позволяет кодировать не по частоте, а просто по наличию (есть/нет слова)
                                 min_df=3,
                                 ngram_range=(2,3),
                                 stop_words=stopwords.words("russian"))


LDA_model = LatentDirichletAllocation(n_components=6,
                                          learning_method='online',
                                          random_state=42,
                                          n_jobs=-1)


N = 1000
panel_x, LDA_model, vectorizer = theme_class(coffee.loc[:N-1, 'text'],
                                             vectorizer_x=vectorizer,
                                             model_type=LDA_model)
print(f'Время обработки = {time.time() - start} секунд')
panel_x

/usr/local/lib/python3.10/dist-packages/ipykernel/ipkernel.py:283: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)
/usr/local/lib/python3.10/dist-packages/pyLDAvis/_prepare.py:244: FutureWarning: In a future version of pandas all arguments of DataFrame.drop except for the argument 'labels' will be keyword-only.
  by='saliency', ascending=False).head(R).drop('saliency', 1)


Log Likelihood -13132.429372091865
Perplexity 3726.788751848393
Время обработки = 10.78743052482605 секунд


PreparedData(topic_coordinates=               x          y  topics  cluster       Freq
topic                                                  
3      10.486069 -28.126221       1        1  18.102762
4     -26.538774   2.977388       2        1  17.792446
5      51.641666  -2.420286       3        1  17.787533
2      -8.489305  47.887604       4        1  16.830224
0      13.443948  12.952583       5        1  15.784594
1      39.764473  44.550087       6        1  13.702441, topic_info=                      Term       Freq      Total Category  logprob  loglift
116       данный заведение  11.000000  11.000000  Default  30.0000  30.0000
579          сделать заказ   6.000000   6.000000  Default  29.0000  29.0000
745             это просто   6.000000   6.000000  Default  28.0000  28.0000
472       порция маленький   4.000000   4.000000  Default  27.0000  27.0000
534  просто отвратительный   4.000000   4.000000  Default  26.0000  26.0000
..                     ...        ...        ...      ...      ...      ...
134        детский комната   1.742711   3.125913   Topic6  -4.8328   1.4033
666           уютный место   1.786825   3.402663   Topic6  -4.8078   1.3435
153         еда нормальный   1.802946   3.747160   Topic6  -4.7989   1.2560
675          хороший место   1.543067   2.983157   Topic6  -4.9545   1.3284
306          место хороший   1.269000   2.595028   Topic6  -5.1500   1.2722

[267 rows x 6 columns], token_table=      Topic      Freq                Term
term                                     
3         1  0.945638  атмосфера приятный
5         5  0.483301        бизнес ланча
7         1  0.530846          блюдо вкус
8         1  0.875246       блюдо вкусный
19        1  0.506838        большой плюс
...     ...       ...                 ...
742       1  0.843892         это приятно
745       3  0.914206          это просто
746       3  0.812262     это просто ужас
751       6  0.514590            это ужас
752       3  0.887016         это ужасный

[284 rows x 3 columns], R=30, lambda_step=0.01, plot_opts={'xlab': 'PC1', 'ylab': 'PC2'}, topic_order=[4, 5, 6, 3, 1, 2])